<a href="https://colab.research.google.com/github/egyptai/Pytorch/blob/main/GatedRecurrentUnit(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install unidecode

In [18]:
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

--2021-06-02 13:45:35--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2021-06-02 13:45:35 (25.2 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [39]:
import torch
import torch.nn as nn
import unidecode
import string
import random
import re
import time, math

num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002


all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars =', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars = 100


In [40]:
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

In [41]:
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [42]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

In [43]:
class RNN(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers =1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.embedding_size = embedding_size

    self.encoder = nn.Embedding(input_size, embedding_size)

    self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
    #self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
    #self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    out = self.encoder(input.view(1,-1))
    out,hidden = self.rnn(out, hidden)
    out = self.decoder(out.view(batch_size, -1))
    return out, hidden

  def init_hidden(self):
    hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
    return hidden

In [44]:
model = RNN(input_size = n_characters, embedding_size = embedding_size, hidden_size = hidden_size,
    output_size = n_characters,
    num_layers=2)

In [45]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [46]:
inp = char_tensor("A")
hidden = model.init_hidden()
out,hidden = model(inp, hidden)

In [47]:
# 임의의 문자(start_str)로 시작하는 길이 200짜리 모방 글을 생성하는 코드입니다.
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 만약 max 값만 쓰는 경우에
        # 생성되는 텍스트가 다 the the the the the 이런식으로 나오기 때문입니다.
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 램덤하게 다음 글자를 뽑아내는 방식으로 자연스러운 텍스트를 생성해냅니다.
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [48]:
for i in range(num_epochs):
  total = char_tensor(random_chunk())
  inp = total[:-1]
  label = total[1:]
  hidden = model.init_hidden()

  loss = torch.tensor([0]).type(torch.FloatTensor)
  optimizer.zero_grad()
  for j in range(chunk_len-1):
      x = inp[j]
      y_=label[j].unsqueeze(0).type(torch.LongTensor)
      y, hidden = model(x, hidden)
      loss += loss_func(y, y_)
  loss.backward()
  optimizer.step()
  if i % 100 == 0:
      print("\n",loss/chunk_len,"\n")
      test()
      print("\n","="*100)


 tensor([4.6188], grad_fn=<DivBackward0>) 

b#r$_yan92cxtKX)%VF2D"GF1Gd
AJ$FApai!Y$>e.A_~I2|S#PevhH7VDO!vsW+m-eJ4K_}:d.BbsT&,y fYVmR1[o]i2pl}6B(.6
VZXB$`tbb X%s6z=36!%GTjZx#&u%I5&8'b(d.eTR;L]+1^]@/5(k<() pO2i0ls,.d^ZB@eq_~ZV$E8qV\C+X6-=N{1ZEpB!

 tensor([2.5672], grad_fn=<DivBackward0>) 

bukdw
Tot brers Tuerill, the moue, sonttelanr hert ' Ir we n oulp Ndor cous kw anpor mer ltsve bmaldt kat east beoren has chak me and pireme ter theu; minee bat toy le, the meat ge nor, goug,, I
Tole s

 tensor([2.3128], grad_fn=<DivBackward0>) 

bfourreem thus line be in'th plint ing otherene weempremy and thite math Aear weale all with onts thall the thas.

GIRINTAAAS:
And anciss yeas thes theill, a lant wisise angees cased, withenein one mea

 tensor([2.0449], grad_fn=<DivBackward0>) 

bis siind with ert Hereadm hey nishy to your me-bat
tuen thtelst ich the dreath wint freare blomire thy cinlere the kere det tho hy or breatomy, sofnel aree s bome main?
Lastog you srmorl ther son'd my

 tensor([2.